In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd
import json
import os
import gzip


In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')



import numpy as np
import pandas as pd

import warnings

from sklearn.utils import shuffle

from keras.utils import to_categorical
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation,Conv1D,MaxPooling1D,Flatten,Dropout
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


In [0]:
def make_train_data():
    data1 = []

    for filename in os.listdir("/content/gdrive/My Drive/input_project"):
        if filename.endswith(".json.gz"): 

            data1.append(filename)
    data=sorted(data1)
    
    
    df=pd.DataFrame()
    for i in range(2):
        df1=getDF(f"/content/gdrive/My Drive/input_project/{data[i]}")
  
        name=[]
        for j in range(len(df1.index)):
            name.append(data[i])
        df1["Name"]=name

        df = df.append(df1, ignore_index=True)




    df=df.drop(["asin","reviewerID","unixReviewTime","reviewTime","summary","reviewerName","helpful"],axis=1)
    df=df.replace(r'^\s*$', np.nan, regex=True).replace('',np.nan)
    df=df.fillna("6")
    df=shuffle(df,replace=False)
    df.reset_index(inplace=True, drop=True)

    df.to_csv('Train_data.csv',index=False)
    

In [0]:
def myTokenizer(sentence):
    tokenizer = RegexpTokenizer("[a-zA-Z]+")
     
    sw = set(stopwords.words('english'))-set(["not"])
    ps = PorterStemmer()

    sentence = tokenizer.tokenize(sentence.lower())
    return [ps.stem(w) for w in sentence if w not in sw]

In [0]:
cv1 = CountVectorizer(tokenizer=myTokenizer)

label_encoder1 = LabelEncoder()
def Clean_sentiment_data(dfx,dfxtt):
    revew=dfx["reviewText"]
    review=revew.values
    review=review.tolist()
    
    revews=dfxtt["reviewText"]
    reviews=revews.values
    reviews=reviews.tolist()
    
    global cv1
    vectorized_corpus = cv1.fit_transform(review)
    
    xtestvec=cv1.transform(reviews)
    
    global label_encoder1
    integer_category = label_encoder1.fit_transform(dfx.overall)
    encoded_y = to_categorical(integer_category)

    integer_category = label_encoder1.fit_transform(dfxtt.overall)
    encoded_yt = to_categorical(integer_category)
    
    return vectorized_corpus,encoded_y,xtestvec,encoded_yt
   

In [0]:
cv2 = CountVectorizer(tokenizer=myTokenizer)

label_encoder2 = LabelEncoder()

def Clean_aspect_data(dfx,dfxtt):
    revew=dfx["reviewText"]
    review=revew.values
    review=review.tolist()
    
    revews=dfxtt["reviewText"]
    reviews=revews.values
    reviews=reviews.tolist()
    
    global cv2

    vectorized_corpus = cv2.fit_transform(review)
    
    xtestvec=cv2.transform(reviews)
    
    global label_encoder2
    integer_category = label_encoder2.fit_transform(dfx.Name)
    encoded_y = to_categorical(integer_category)

    integer_category = label_encoder2.fit_transform(dfxtt.Name)
    encoded_yt = to_categorical(integer_category)
    
    return vectorized_corpus,encoded_y,xtestvec,encoded_yt
   

In [0]:
cv3 = CountVectorizer(tokenizer=myTokenizer)

label_encoder3 = LabelEncoder()

def Clean_rating_data(dfx,dfxtt):
    revew=dfx["reviewText"]
    review=revew.values
    review=review.tolist()
    
    revews=dfxtt["reviewText"]
    reviews=revews.values
    reviews=reviews.tolist()
    
    global cv3

    vectorized_corpus = cv3.fit_transform(review)
    
    xtestvec=cv3.transform(reviews)
    
    global label_encoder3
    integer_category = label_encoder3.fit_transform(dfx.overall)
    encoded_y = to_categorical(integer_category)

    integer_category = label_encoder3.fit_transform(dfxtt.overall)
    encoded_yt = to_categorical(integer_category)
    
    return vectorized_corpus,encoded_y,xtestvec,encoded_yt
   

In [0]:
def Clean_sentiment_test_data(dfxt):
    revew=dfxt["reviewText"]
    review=revew.values
    review=review.tolist()
    global cv1
    xtestvec=cv1.transform(review)
    
    return xtestvec
    

In [0]:
def Clean_aspect_test_data(dfxt):
    revew=dfxt["reviewText"]
    review=revew.values
    review=review.tolist()
    global cv2
    xtestvec=cv2.transform(review)
    
    return xtestvec
    

In [0]:
def Clean_rating_test_data(dfxt):
    revew=dfxt["reviewText"]
    review=revew.values
    review=review.tolist()
    global cv3
    xtestvec=cv3.transform(review)
    
    return xtestvec
    

In [0]:
model1 = Sequential()

def Train_sentiment_data(dfx,dfxtt):
    
    X,Y,xtestvec,encoded_yt=Clean_sentiment_data(dfx,dfxtt)
    
    

    #XTrain= preprocess_data(X)
    #xtestvec=preprocess_data(xtestvec)
    global model
    
    model1.add(Dense(512, input_shape=(X.shape[1],), activation='relu'))
    model1.add((Dense(256, activation='relu')))
    model1.add((Dense(128, activation='relu')))
    model1.add(Dense(2,activation='softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    model1.fit(X,Y, epochs=20,validation_data=(xtestvec,encoded_yt))
    return xtestvec,encoded_yt
    

In [0]:
model2 = Sequential()

def Train_aspect_data(dfx,dfxtt):
    
    X,Y,xtestvec,encoded_yt=Clean_aspect_data(dfx,dfxtt)
    
    

    #XTrain= preprocess_data(X)
    #xtestvec=preprocess_data(xtestvec)
    global model2
    
    model2.add(Dense(512, input_shape=(X.shape[1],), activation='relu'))
    model2.add((Dense(256, activation='relu')))
    model2.add((Dense(128, activation='relu')))
    model2.add(Dense(2,activation='softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    model2.fit(X,Y, epochs=10,validation_data=(xtestvec,encoded_yt))
    

In [0]:
model3 = Sequential()

def Train_rating_data(dfx,dfxtt):
    
    X,Y,xtestvec,encoded_yt=Clean_rating_data(dfx,dfxtt)
    
    

    #XTrain= preprocess_data(X)
    #xtestvec=preprocess_data(xtestvec)
    global model3
    
    model3.add(Dense(512, input_shape=(X.shape[1],), activation='relu'))
    model3.add((Dense(256, activation='relu')))
    model3.add((Dense(128, activation='relu')))
    model3.add(Dense(5,activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
    model3.fit(X,Y, epochs=10,validation_data=(xtestvec,encoded_yt))
    

In [0]:
def Train_for_sentiment():
  dfx=pd.read_csv("Train_data.csv")
  bigdata=dfx
  '''bigdata.overall[(3<bigdata.overall) & (bigdata.overall<=5)]="POSTIVE"
  bigdata.overall[bigdata.overall==3]="NEUTRAL"
  bigdata.overall[bigdata.overall<3]="NEGATIVE"'''
  for i in range(len(bigdata.overall)):
    if (3<=bigdata.overall[i]) & (bigdata.overall[i]<=5):
      bigdata.overall[i]="POSITIVE"
    elif bigdata.overall[i]<3:
      bigdata.overall[i]="NEGATIVE"
  dfxt=bigdata.iloc[:60000,:]

  dfxtt=bigdata.iloc[60000:,:]
  xtestvec,encoded_yt=Train_sentiment_data(dfxt,dfxtt)
  return xtestvec,encoded_yt

In [0]:
def Train_for_category():
  dfx=pd.read_csv("Train_data.csv")
  bigdata=dfx
  dfxt=bigdata.iloc[:60000,:]

  dfxtt=bigdata.iloc[60000:,:]
  Train_aspect_data(dfxt,dfxtt)

In [0]:
def Train_for_rating():
  dfx=pd.read_csv("Train_data.csv")
  bigdata=dfx
  dfxt=bigdata.iloc[:60000,:]

  dfxtt=bigdata.iloc[60000:,:]
  Train_rating_data(dfxt,dfxtt)

In [0]:
def Get_the_sentiment():
    fb = open("abcd.txt", 'w')
    

    with open("input.txt", 'r') as f:
      for line in f:
        fb.write(line.replace(",", "."))
    #print(line.replace(",", "."))
        fb.write("\n")
    fb.close()
    dfxt=pd.read_csv('abcd.txt', header=None)

    dfxt.columns = ["reviewText"]
    
    X_test=Clean_sentiment_test_data(dfxt)
    global model1
    test_category = label_encoder1.inverse_transform(model1.predict_classes(X_test))
    probab=model1.predict_proba(X_test)
    
    '''print(f"Probability of All beauty is : {probab[0][0]}")
    print(f"Probability of Grocery is    : {probab[0][1]} ")'''
    for i in range(len(test_category)):
      print(f"The overall rating is {test_category[i]}")
    
   
    
    

In [0]:
def Get_the_category():
    
    fb = open("abcd.txt", 'w')

    with open("input.txt", 'r') as f:
      for line in f:
        fb.write(line.replace(",", "."))
    #print(line.replace(",", "."))
        fb.write("\n")
    fb.close()
    dfxt=pd.read_csv('abcd.txt', header=None)

    dfxt.columns = ["reviewText"]
    
    X_test=Clean_aspect_test_data(dfxt)
    global model2
    test_category = label_encoder2.inverse_transform(model2.predict_classes(X_test))
    probab=model2.predict_proba(X_test)
    
    '''print(f"Probability of All beauty is : {probab[0][0]}")
    print(f"Probability of Grocery is    : {probab[0][1]} ")'''
    for i in range(len(test_category)):
      test_category[i]=test_category[i].replace(".json.gz","")
      print(f"This review belong to {test_category[i]}")
    
  
    
    

In [0]:
def Get_the_rating():
    
    fb = open("abcd.txt", 'w')

    with open("input.txt", 'r') as f:
      for line in f:
        fb.write(line.replace(",", "."))
    #print(line.replace(",", "."))
        fb.write("\n")
    fb.close()
    dfxt=pd.read_csv('abcd.txt', header=None)

    dfxt.columns = ["reviewText"]
    
    X_test=Clean_rating_test_data(dfxt)
    global model3
    test_category = label_encoder3.inverse_transform(model3.predict_classes(X_test))
    probab=model3.predict_proba(X_test)
    
    '''print(f"Probability of All beauty is : {probab[0][0]}")
    print(f"Probability of Grocery is    : {probab[0][1]} ")'''
    for i in range(len(test_category)):
      print(f"The overall rating is {test_category[i]}")
    
    
    
    

In [0]:
def Get_sentiment():
    fb = open("abcd.txt", 'w')
    

    with open("input.txt", 'r') as f:
      for line in f:
        fb.write(line.replace(",", "."))
    #print(line.replace(",", "."))
        fb.write("\n")
    fb.close()
    dfxt=pd.read_csv('abcd.txt', header=None)

    dfxt.columns = ["reviewText"]
    
    X_test=Clean_sentiment_test_data(dfxt)
    global model1
    test_category = label_encoder1.inverse_transform(model1.predict_classes(X_test))
    probab=model1.predict_proba(X_test)
    
    '''print(f"Probability of All beauty is : {probab[0][0]}")
    print(f"Probability of Grocery is    : {probab[0][1]} ")
    for i in range(len(test_category)):
      if test_category[i]==1:
        print(f"This review is POSITIVE")
      else:
        print("This review is NEGATIVE")'''
    return test_category
   
    
    

In [0]:
def Get_category():
    
    fb = open("abcd.txt", 'w')

    with open("input.txt", 'r') as f:
      for line in f:
        fb.write(line.replace(",", "."))
    #print(line.replace(",", "."))
        fb.write("\n")
    fb.close()
    dfxt=pd.read_csv('abcd.txt', header=None)

    dfxt.columns = ["reviewText"]
    
    X_test=Clean_aspect_test_data(dfxt)
    global model2
    test_category = label_encoder2.inverse_transform(model2.predict_classes(X_test))
    probab=model2.predict_proba(X_test)
    
    '''print(f"Probability of All beauty is : {probab[0][0]}")
    print(f"Probability of Grocery is    : {probab[0][1]} ")
    for i in range(len(test_category)):
      print(f"This review belong to {test_category[i]}")'''
    return test_category
    
    
    
    
    

In [0]:
def Get_rating():
    
    fb = open("abcd.txt", 'w')

    with open("input.txt", 'r') as f:
      for line in f:
        fb.write(line.replace(",", "."))
    #print(line.replace(",", "."))
        fb.write("\n")
    fb.close()
    dfxt=pd.read_csv('abcd.txt', header=None)

    dfxt.columns = ["reviewText"]
    
    X_test=Clean_rating_test_data(dfxt)
    global model3
    test_category = label_encoder3.inverse_transform(model3.predict_classes(X_test))
    probab=model3.predict_proba(X_test)
    
    '''print(f"Probability of All beauty is : {probab[0][0]}")
    print(f"Probability of Grocery is    : {probab[0][1]} ")
    for i in range(len(test_category)):
      print(f"The overall rating is {test_category[i]}")'''
    return test_category
    
    
    

In [0]:
def Result():
  S=Get_sentiment()
  C=Get_category()
  R=Get_rating()

  for i in range(len(S)):
    if (R[i] in [1.0,2.0]) & (S[i]=="POSITIVE"):
      print("This review is SOMEWHAT POSITIVE")
      C[i]=C[i].replace(".json.gz","")
      print(f"It belongs to belongs to {C[i]}")
      print(f"It's overall rating is {R[i]} \n")
    elif (R[i] in [4.0,5.0]) & (S[i]=="NEGATIVE"):
      print("This review is SOMEWHAT NEGATIVE")
      C[i]=C[i].replace(".json.gz","")
      print(f"It belongs to belongs to {C[i]}")
      print(f"It's overall rating is {R[i]} \n")
    else: 
        print(f"This review is {S[i]} ")
        C[i]=C[i].replace(".json.gz","")
        print(f"It belongs to belongs to {C[i]}")
        print(f"It's overall rating is {R[i]} \n ")



Classification starts

In [0]:
make_train_data()

In [0]:
Train_for_sentiment()





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 11973 samples
Epoch 1/20





60000/60000 [==============================] - 34s 559us/step - loss: 0.2696 - acc: 0.8957 - val_loss: 0.2355 - val_acc: 0.9060
Epoch 2/20
60000/60000 [==============================] - 24s 397us/step - loss: 0.1565 - acc: 0.9372 - val_loss: 0.2669 - val_acc: 0.8942
Epoch 3/20
60000/60000 [==============================] - 23s 389us/step - loss: 0.0415 - acc: 0.9860 - val_loss: 0.4539 - val_acc: 0.8987
Epoch 4/20
60000/60000 [==============================] - 23s 383us/step - loss: 0.0090 - acc: 0.9974 - val_loss: 0.5488 - val_acc: 0.9013
Epoch 5/20
60000/60000 [==============================] - 23s 386us/step - loss: 0.0075 - acc: 0.9984 - val_loss: 0.6636 - val_acc: 0.9002
Epoch 6/20
60000/60000 [==============================] - 23s 382us/step - loss: 0.0064 - acc: 0.9987 - val_loss: 0.8450 - val_acc: 0.9008
Epoch 7/20

(<11973x33630 sparse matrix of type '<class 'numpy.int64'>'
 	with 470537 stored elements in Compressed Sparse Row format>,
 array([[1., 0.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [1., 0.],
        [0., 1.]], dtype=float32))

In [0]:
Train_for_category()

Train on 60000 samples, validate on 11973 samples
Epoch 1/10
60000/60000 [==============================] - 23s 390us/step - loss: 0.0409 - acc: 0.9884 - val_loss: 0.0409 - val_acc: 0.9916
Epoch 2/10
60000/60000 [==============================] - 23s 382us/step - loss: 0.0113 - acc: 0.9969 - val_loss: 0.0454 - val_acc: 0.9921
Epoch 3/10
60000/60000 [==============================] - 23s 380us/step - loss: 0.0072 - acc: 0.9986 - val_loss: 0.0688 - val_acc: 0.9911
Epoch 4/10
60000/60000 [==============================] - 23s 379us/step - loss: 0.0048 - acc: 0.9991 - val_loss: 0.0718 - val_acc: 0.9914
Epoch 5/10
60000/60000 [==============================] - 23s 378us/step - loss: 0.0060 - acc: 0.9992 - val_loss: 0.0825 - val_acc: 0.9911
Epoch 6/10
60000/60000 [==============================] - 23s 380us/step - loss: 0.0041 - acc: 0.9993 - val_loss: 0.0731 - val_acc: 0.9916
Epoch 7/10
60000/60000 [==============================] - 23s 377us/step - loss: 0.0027 - acc: 0.9996 - val_loss: 0.

In [0]:
Train_for_rating()

Train on 60000 samples, validate on 11973 samples
Epoch 1/10
60000/60000 [==============================] - 24s 401us/step - loss: 1.0023 - acc: 0.5917 - val_loss: 0.9367 - val_acc: 0.6170
Epoch 2/10
60000/60000 [==============================] - 23s 387us/step - loss: 0.7568 - acc: 0.6894 - val_loss: 0.9982 - val_acc: 0.6085
Epoch 3/10
60000/60000 [==============================] - 23s 376us/step - loss: 0.4286 - acc: 0.8313 - val_loss: 1.3358 - val_acc: 0.5872
Epoch 4/10
60000/60000 [==============================] - 23s 378us/step - loss: 0.1700 - acc: 0.9403 - val_loss: 1.9019 - val_acc: 0.5751
Epoch 5/10
60000/60000 [==============================] - 23s 378us/step - loss: 0.0723 - acc: 0.9776 - val_loss: 2.3991 - val_acc: 0.5743
Epoch 6/10
60000/60000 [==============================] - 23s 377us/step - loss: 0.0451 - acc: 0.9856 - val_loss: 2.7781 - val_acc: 0.5715
Epoch 7/10
60000/60000 [==============================] - 23s 379us/step - loss: 0.0381 - acc: 0.9883 - val_loss: 2.

In [0]:
Get_the_sentiment()

The overall rating is NEGATIVE
The overall rating is NEGATIVE
The overall rating is POSITIVE


In [0]:
Get_the_category()

This review belong to reviews_All_Beauty
This review belong to reviews_Grocery_and_Gourmet_Food_10
This review belong to reviews_Grocery_and_Gourmet_Food_10


In [0]:
Get_the_rating()

The overall rating is 1.0
The overall rating is 1.0
The overall rating is 5.0


In [0]:
Result()



This review is NEGATIVE 
It belongs to belongs to reviews_All_Beauty
It's overall rating is 1.0 
 
This review is NEGATIVE 
It belongs to belongs to reviews_Grocery_and_Gourmet_Food_10
It's overall rating is 1.0 
 
This review is POSITIVE 
It belongs to belongs to reviews_Grocery_and_Gourmet_Food_10
It's overall rating is 5.0 
 
